In [2]:
import pandas as pd 
import numpy as np
import matplotlib as plt
import nltk 
import gzip
from scipy.sparse import csr_matrix


In [3]:
from sklearn.metrics.pairwise import cosine_similarity

In [4]:
df_ratings1 = pd.read_csv("ratings_Patio_Lawn_and_Garden.csv")
print(df_ratings1.shape)
df_ratings = df_ratings1[:1000]
print(df_ratings.shape)


(993489, 4)
(1000, 4)


In [5]:
# To read the reviews data 

def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield eval(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

df_reviews = getDF("reviews_Patio_Lawn_and_Garden_5.json.gz")


In [6]:
df_ratings.head()

,A2VNYWOPJ13AFP,0981850006,5.0,1259798400
0,A20DWVV8HML3AW,0981850006,5.0,1371081600
1,A3RVP3YBYYOPRH,0981850006,5.0,1257984000
2,A28XY55TP3Q90O,0981850006,5.0,1314144000
3,A3VZW1BGUQO0V3,0981850006,5.0,1308268800
4,A2R9T5D7UVQZB0,0981850006,5.0,1253577600


In [7]:
df_ratings = df_ratings.rename(columns={"A2VNYWOPJ13AFP":"UserID","0981850006":"ObjectID","5.0":"Rating","1259798400":"UnixTime"})
df_ratings.head()

,UserID,ObjectID,Rating,UnixTime
0,A20DWVV8HML3AW,0981850006,5.0,1371081600
1,A3RVP3YBYYOPRH,0981850006,5.0,1257984000
2,A28XY55TP3Q90O,0981850006,5.0,1314144000
3,A3VZW1BGUQO0V3,0981850006,5.0,1308268800
4,A2R9T5D7UVQZB0,0981850006,5.0,1253577600


In [8]:
df_ratings["ObjectID"] = df_ratings["ObjectID"].astype("string")


In [9]:
df_ratings["UserID"] = df_ratings["UserID"].astype("string")

### Only needed data for recommendation system is User ID and objectID

Step 1: Create a sparse matrix out of the features you require (user rating, correlation, usw.)

Step 2: Can you Nearest Neighbours to get the distances of objects in this sparse matrix w.r.t to any reference provided. 

In [10]:
df_ratings.head()

,UserID,ObjectID,Rating,UnixTime
0,A20DWVV8HML3AW,0981850006,5.0,1371081600
1,A3RVP3YBYYOPRH,0981850006,5.0,1257984000
2,A28XY55TP3Q90O,0981850006,5.0,1314144000
3,A3VZW1BGUQO0V3,0981850006,5.0,1308268800
4,A2R9T5D7UVQZB0,0981850006,5.0,1253577600


In [11]:
df_ratings["UserID"][0]

'A20DWVV8HML3AW'

In [12]:
df_ratings = df_ratings.drop("UnixTime",axis=1)


In [13]:
df_ratings.head()
print(df_ratings.shape)
df_ratings = df_ratings.drop_duplicates()
print(df_ratings.shape)

(1000, 3)
(1000, 3)


In [14]:
df_pivot = (df_ratings.pivot(index="UserID",columns="ObjectID",values="Rating").fillna(0))
sparse_matrix = csr_matrix(df_pivot.values)



Collaborative Filtering works by getting a similarity score,

Here we use cosine similarity

In [15]:
from sklearn.neighbors import NearestNeighbors
nn = NearestNeighbors(metric="cosine",algorithm="brute")
nn.fit(sparse_matrix)

NearestNeighbors(algorithm='brute', metric='cosine')

### Getting Recommendations

In [17]:


random_user = np.random.choice(df_pivot.shape[0])

indices, distance = nn.kneighbors(df_pivot.iloc[random_user,:].values.reshape(-1,1),n_neighbors=6)

ValueError: X has 1 features, but NearestNeighbors is expecting 86 features as input.